In [1]:
import gc
import math
import random

import torch
import torch.nn as nn

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from tqdm import trange
from pathlib import Path

d:\code\dementiakiller\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# CONFIG

# Training
LR = 0.001
MELS = 50 # FIXED due to preprocessing
BS = 32 # DO NOT CHANGE ATM fixed to numpy folder as each bach is saved as a numpy array 

DEVICE = 'cuda'

CHARSET = " .,!abcdefghijklmnopqrstuvwxyz"


 # TODO: Clean this up
FOLDER = Path('data/LJSpeech-1.1/wavs')

path_numpy = FOLDER.parent / Path('numpy')
path_data = path_numpy / Path('data')
path_labels = path_numpy / Path('labels')
path_input = path_numpy / Path('input_lengths')
path_target = path_numpy / Path('target_lengths')

path_models = Path('models')

path_models.mkdir(exist_ok=True)

In [3]:
# Model

# TODO: CLEAN UP THIS!
C = 16 # BUG: Can we risk clipping for n value of C?
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()

        self.conv = nn.Sequential(
                                nn.Conv2d(1,C, kernel_size=3, stride=1, padding='same'),
                                nn.ReLU(),
                                nn.BatchNorm2d(C),
                                nn.Conv2d(C,C*2, kernel_size=3, stride=1, padding='same'),
                                nn.ReLU(),
                                nn.BatchNorm2d(C*2),
                                )                          
                                

        self.ln1 = nn.Sequential(
                                nn.Linear((C*2)*50,600),
                                nn.ReLU(),                                
                                nn.Linear(600,300),
                                nn.ReLU(),
                                )

        self.gru = nn.GRU(300, 300, batch_first=False)
        self.ln2 = nn.Sequential(
                        nn.Linear(300, 150),
                        nn.ReLU(),
                        nn.Linear(150, 75),
                        nn.ReLU(),
                        nn.Linear(75,len(CHARSET)),
                         )

    def forward(self, x):
        # TODO: Use different implementation/Input when using CONV?
        # TODO: When using conv should we change input ? Input ATM: (Time,Batch,Mel/Freq) 
        # TODO: CLEAN UP
        x = x.reshape(x.shape[1],1,-1,MELS)
        #print(x.shape)
        x = self.conv(x)
        #print(x.shape)
        x = x.permute(0, 2, 1, 3)
        #print("dis" + str(x.shape))
        x = x.reshape(x.shape[0],-1,(C*2)*MELS)
        x = x.permute(1,0,2)
        #print(x.shape)
        x = self.ln1(x)
        #print(x.shape)
        x = nn.functional.relu(self.gru(x)[0])
        #print(x.shape)
        x = self.ln2(x)
        #print(x.shape)
        #  Output (Time,Batch,Mel/Freq) -> Convention when using CTC loss 
        x = torch.nn.functional.log_softmax(x, dim=2)
        return x

model = Net().to(DEVICE)
model
"""

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()

        self.ln1 = nn.Sequential(
                                nn.Linear(50,100),
                                nn.ReLU(),                                
                                nn.Linear(100,250),
                                nn.ReLU(),
                                nn.Linear(250,500),
                                nn.ReLU(),
                                nn.Linear(500,500),
                                nn.ReLU(),
                                )

        self.gru = nn.GRU(500, 500, batch_first=False)
        self.ln2 = nn.Sequential(
                        nn.Linear(500, 250),
                        nn.ReLU(),
                        nn.Linear(250, 75),
                        nn.ReLU(),
                        nn.Linear(75,len(CHARSET)),
                         )

    def forward(self, x):
        x = self.ln1(x)
        x = nn.functional.relu(self.gru(x)[0])
        x = self.ln2(x)
        x = torch.nn.functional.log_softmax(x, dim=2)
        return x

model = Net().to(DEVICE)
"""

'\n\nclass Net(nn.Module):\n\n    def __init__(self):\n        super(Net, self).__init__()\n\n        self.ln1 = nn.Sequential(\n                                nn.Linear(50,100),\n                                nn.ReLU(),                                \n                                nn.Linear(100,250),\n                                nn.ReLU(),\n                                nn.Linear(250,500),\n                                nn.ReLU(),\n                                nn.Linear(500,500),\n                                nn.ReLU(),\n                                )\n\n        self.gru = nn.GRU(500, 500, batch_first=False)\n        self.ln2 = nn.Sequential(\n                        nn.Linear(500, 250),\n                        nn.ReLU(),\n                        nn.Linear(250, 75),\n                        nn.ReLU(),\n                        nn.Linear(75,len(CHARSET)),\n                         )\n\n    def forward(self, x):\n        x = self.ln1(x)\n        x = nn.functional.

In [4]:
# How are shapes with CTC loss working? 
# INPUT [time,batch,mels]
inn = torch.rand(1114,32,50, device=DEVICE)
print(f"Input shape: {inn.shape}")
test = model(inn)
#
print(f"Output shape: {test.shape}")
# OUTPUT [time,batch,len(CHARSET)]

Input shape: torch.Size([1114, 32, 50])
Output shape: torch.Size([1114, 32, 30])


In [5]:
# Criterion (Loss function) and Optimizer
criterion  = nn.CTCLoss(reduction='mean', zero_infinity=True)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

In [6]:
# TODO: we need to shuffle, currently every batch and epoch are the same
# TODO: Normalize data?
# TODO: Char mappings how are padding handled vs space 

EPOCHS = 500

traning_loss = []

t = trange(EPOCHS)
import time
for epoch in t:
    for batch in range(408):

        #start = time.time()
        batch = random.randint(0, 408) # BUG: HAX: TODO: BAD
        inputs = np.load(path_data / Path(str(batch) + '.npy'))
        labels = np.load(path_labels / Path(str(batch) + '.npy'))
        input_lengths = np.load(path_input / Path(str(batch) + '.npy'))
        target_lengths = np.load(path_target / Path(str(batch) + '.npy'))

        inputs = torch.tensor(inputs).reshape(-1,BS,50).float().to(DEVICE)
        labels = torch.tensor(labels).float().to(DEVICE)
        input_lengths = torch.tensor(input_lengths)
        target_lengths = torch.tensor(target_lengths)
        
        # NOTE: 10s to load this way per epoch for LJSpeech
        #end = time.time()
        #print(end-start)
        
        #inputs = torch.nn.functional.normalize(inputs)

        optimizer.zero_grad() # Reset gradient to zero

        # Forward
        output = model(inputs) 

        # Compute loss
        loss = criterion(output, labels, input_lengths,target_lengths)

        # Update loss
        loss.backward()

        # optimizer
        optimizer.step()
        if batch % 10 == 0:
            t.set_description(f"Loss: {loss.item()}")

        del inputs
        del labels
    
    if True:
        torch.save(model.state_dict(), f"models/{str(epoch)}-c.pt")
    
    traning_loss.append(loss.item())
    


Loss: 2.5977869033813477:  32%|███▏      | 158/500 [2:06:09<4:38:28, 48.85s/it]

In [ ]:
plt.plot(traning_loss)

In [ ]:
# TEST
TEST = 23 
TEST2 = 0

data = np.load(path_data / Path(str(TEST) + '.npy'))
data = torch.tensor(data[TEST2]).float()
data = data[:,:,None].permute(1,2,0).to(DEVICE)
print(data.shape)

# Run the model
output = model(data)
output = output[:, 0, :].argmax(dim=1)

estimate = []
for char in output:
    if char != 0:
        estimate.append(CHARSET[char-1])
print(''.join(estimate))